!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!This script contains maskign code before standardisation. however standardisation of inputs and targets was performed before the interpoléated baselines were generated via CDO for bilinear and bicubic and then masked so that they only retain the Swiss Domain

Workflow : Bilinear interpolation, bicubic interpolation, Masking to retain only the swiss domain, splitting into train, test val, gridded RMSE R2, pooled RMSE, R2

In [2]:
import sys
sys.path.append("../../Scripts/Functions")
import xarray as xr
import numpy as np

In [3]:
from dask.distributed import Client
client= Client()
print(client)

<Client: 'tcp://127.0.0.1:44725' processes=4 threads=4, memory=503.49 GiB>


installed dask distributed, and used all 4 CPU cores, computing time reduced to 30 seconds per line!!!

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxCHECKING SHAPExxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

FEATURES : CHECKIGN SHAPES

In [4]:

features_ds_precip = xr.open_dataset("../../data/Features/files/bicubic_rhiresd.nc")["RhiresD"]
features_ds_precip.shape

(19358, 240, 370)

In [5]:
features_ds_tabsd= xr.open_dataset("../../data/Features/files/bicubic_tabsd.nc")["TabsD"]
features_ds_tabsd.shape

(19358, 240, 370)

In [6]:
features_ds_tmax= xr.open_dataset("../../data/Features/files/bicubic_tmax.nc")["TmaxD"]
features_ds_tmax.shape

(19358, 240, 370)

In [ ]:
features_ds_tmin= xr.open_dataset("../../data/Features/files/bicubic_tmin.nc")["TminD"]
features_ds_tmin.shape

(19358, 240, 370)

CHECKING SHAPES OF TARGETS

In [7]:
targets_ds_precip = xr.open_dataset("../../data/Targets/files/RhiresD_wet_1971_2023.nc")["RhiresD"]
targets_ds_precip.shape

(19358, 240, 370)

In [ ]:
targets_ds_tabsd= xr.open_dataset("../../data/Targets/files/TabsD_wet_1971_2023.nc")["TabsD"]
targets_ds_tabsd.shape

(19358, 240, 370)

In [ ]:
targets_ds_tmax= xr.open_dataset("../../data/Targets/files/TmaxD_wet_1971_2023.nc")["TmaxD"]

In [ ]:
targets_ds_tmin= xr.open_dataset("../../data/Targets/files/TminD_wet_1971_2023.nc")["TminD"]
targets_ds_tmin.shape

(19358, 240, 370)

SHAPES MATCH!!!!!!

The entire dataset will be divided into tain, test val for both bilinear and bicubic interpolation for calculating standardisation metric etc


All splits performed according to the function written in Functions/Train_Test_Val.py

In [4]:
from Train_Test_Val import split_by_decade

Splitting features

RhiresD, TabsD, Tmin, Tmax

Example Usage

In [ ]:

train, val, test = split_by_decade(
    features_ds_precip,
    "../../data/Features/Split_Data",
    "rhiresd_features"
)


Splitting targets along the same lines

Example Usage

In [ ]:
targets_ds_tmax = xr.open_dataset(
    "../../data/Targets/files/TmaxD_wet_1971_2023.nc",
    chunks={"time": 365})["TmaxD"]

train, val, test = split_by_decade(
    targets_ds_tmax,
    "../../data/Features/Split_Data",
    "tmax_targets"
)

Also splittign the targets along the very same lines usign the reproducible split

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxDoing the same train test val split for bilinear nterpolation xxxxxxxxx

Standardisation code

Standardisation has to be performed avoiding data leakage. Hence, the train -test contamination has to be avoided. 

Can be done using statistics only for the training set

The mean, std, min, max and range of the training set will then be used to perform scaling for the test set as well. 

xxxxxxxxxxxxPerforming standardisation for the training set for both precip and temperaturexxxxx. Training will be performed on the bicubically interpolated dataset for now

In [5]:
from Standardise import standardise

Standardisation proceeds in the following steps

1. Features : 
Precipitation
TabsD
TminD
TmaxD

2. Targets:
Precipitation
TabsD
TminD
TmaxD

Precip is scaled using min-max scaling, while temperatures of all kinds are scaled using their empirical mean and std (of the training set in both cases to prevent data leakage)

1. Features

Precipitation train set


In [9]:
input_path="../../data/Features/Split_Data/Train/rhiresd_features_train.nc"
output_path = "../../data/Features/Split_Data_Normalised/SCALED_rhiresd_features_train.nc"
var = "RhiresD"
precip_min, precip_max = standardise(input_path, output_path, var)

Normalised/scaled variable saved to ../../data/Features/Split_Data_Normalised/SCALED_rhiresd_features_train.nc


TabsD train set

In [ ]:
tabsd_mean, tabsd_std = standardise(
    input_path="../../data/Features/Split_Data/Train/tabsd_features_train.nc",
    output_path="../../data/Features/Split_Data_Normalised/SCALED_tabsd_features_train.nc",
    var="TabsD"
)

Tmax train set

In [11]:
tmax_mean, tmax_std = standardise(
    input_path="../../data/Features/Split_Data/Train/tmax_features_train.nc",
    output_path="../../data/Features/Split_Data_Normalised/SCALED_tmax_features_train.nc",
    var="TmaxD"
)

Normalised/scaled variable saved to ../../data/Features/Split_Data_Normalised/SCALED_tmax_features_train.nc


Tmin train set

In [12]:
tmin_mean, tmin_std = standardise(
    input_path="../../data/Features/Split_Data/Train/tmin_features_train.nc",
    output_path="../../data/Features/Split_Data_Normalised/SCALED_tmin_features_train.nc",
    var="TminD"
)

Normalised/scaled variable saved to ../../data/Features/Split_Data_Normalised/SCALED_tmin_features_train.nc


Using above mean, max, std and min from temp and precip to standardise the vcalidation and testing data to avoid data leakage

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxPRECIPITATIONxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

Validation set : Precip, TabsD, TmaxD and TminD, (IN THAT ORDER)

In [ ]:
# Validation features :Precip
input_path = "../../data/Features/Split_Data/Val/rhiresd_features_val.nc"
output_path = "../../data/Features/Split_Data_Normalised/Val/SCALED_rhiresd_features_val.nc"
var = "RhiresD"
standardise(input_path, output_path, var, min=precip_min, max=precip_max)

In [ ]:
input_path = "../../data/Features/Split_Data/Val/tabsd_features_val.nc"
output_path = "../../data/Features/Split_Data_Normalised/Val/SCALED_tabsd_features_val.nc"
var = "TabsD"
standardise(input_path, output_path, var, mean=tabsd_mean, std=tabsd_std)

In [ ]:
input_path = "../../data/Features/Split_Data/Val/tmin_features_val.nc"
output_path = "../../data/Features/Split_Data_Normalised/Val/SCALED_tmin_features_val.nc"
var = "TminD"
standardise(input_path, output_path, var, mean=tmin_mean, std=tmin_std)

In [ ]:
input_path = "../../data/Features/Split_Data/Val/tmax_features_val.nc"
output_path = "../../data/Features/Split_Data_Normalised/Val/SCALED_tmax_features_val.nc"
var = "TmaxD"
standardise(input_path, output_path, var, mean=tmax_mean, std=tmax_std)

TEST SET : Precip,TabsD, TminD, TmaxD (in that order )

In [ ]:
input_path = "../../data/Features/Split_Data/Test/rhiresd_features_test.nc"
output_path = "../../data/Features/Split_Data_Normalised/Test/SCALED_rhiresd_features_test.nc"
var = "RhiresD"
standardise(input_path, output_path, var, min=precip_min, max=precip_max)

In [ ]:
input_path = "../../data/Features/Split_Data/Test/tabsd_features_test.nc"
output_path = "../../data/Features/Split_Data_Normalised/Test/SCALED_tabsd_features_test.nc"
var = "TabsD"
standardise(input_path, output_path, var, mean=tabsd_mean, std=tabsd_std)

In [ ]:
input_path = "../../data/Features/Split_Data/Test/tmin_features_test.nc"
output_path = "../../data/Features/Split_Data_Normalised/Test/SCALED_tmin_features_test.nc"
var = "TminD"
standardise(input_path, output_path, var, mean=tmin_mean, std=tmin_std)

In [ ]:
input_path = "../../data/Features/Split_Data/Test/tmax_features_test.nc"
output_path = "../../data/Features/Split_Data_Normalised/Test/SCALED_tmax_features_test.nc"
var = "TmaxD"
standardise(input_path, output_path, var, mean=tmax_mean, std=tmax_std)

NORMALISING THE TARGETS USING THE SAME SCHEMA OUTLINED ABOVE 

In [22]:
target_precip_min, target_precip_max = standardise(
    input_path="../../data/Targets/Split_Data/Train/rhiresd_targets_train.nc",
    output_path="../../data/Targets/Split_Data_Normalised/Train/SCALED_rhiresd_targets_train.nc",
    var="RhiresD"
)

Normalised/scaled variable saved to ../../data/Targets/Split_Data_Normalised/Train/SCALED_rhiresd_targets_train.nc


In [ ]:
target_tabsd_mean, target_tabsd_std = standardise(
    input_path="../../data/Targets/Split_Data/Train/tabsd_targets_train.nc",
    output_path="../../data/Targets/Split_Data_Normalised/Train/SCALED_tabsd_targets_train.nc",
    var="TabsD"
)

In [24]:
target_tmin_mean, target_tmin_std = standardise(
    input_path="../../data/Targets/Split_Data/Train/tmin_targets_train.nc",
    output_path="../../data/Targets/Split_Data_Normalised/Train/SCALED_tmin_targets_train.nc",
    var="TminD"
)

Normalised/scaled variable saved to ../../data/Targets/Split_Data_Normalised/Train/SCALED_tmin_targets_train.nc


In [25]:
target_tmax_mean, target_tmax_std = standardise(
    input_path="../../data/Targets/Split_Data/Train/tmax_targets_train.nc",
    output_path="../../data/Targets/Split_Data_Normalised/Train/SCALED_tmax_targets_train.nc",
    var="TmaxD"
)

Normalised/scaled variable saved to ../../data/Targets/Split_Data_Normalised/Train/SCALED_tmax_targets_train.nc


Standardising the Val set of the features 

In [ ]:
input_path = "../../data/Targets/Split_Data/Val/rhiresd_targets_val.nc"
output_path = "../../data/Targets/Split_Data_Normalised/Val/SCALED_rhiresd_targets_val.nc"
var = "RhiresD"
standardise(input_path, output_path, var, min=target_precip_min, max=target_precip_max)

In [ ]:
input_path = "../../data/Targets/Split_Data/Val/tabsd_targets_val.nc"
output_path = "../../data/Targets/Split_Data_Normalised/Val/SCALED_tabsd_targets_val.nc"
var = "TabsD"
standardise(input_path, output_path, var, mean=target_tabsd_mean, max=target_tabsd_std)

In [ ]:
input_path = "../../data/Targets/Split_Data/Val/tmin_targets_val.nc"
output_path = "../../data/Targets/Split_Data_Normalised/Val/SCALED_tmin_targets_val.nc"
var = "TminD"
standardise(input_path, output_path, var, mean=target_tmin_mean, max=target_tmin_std)

In [ ]:
input_path = "../../data/Targets/Split_Data/Val/tmax_targets_val.nc"
output_path = "../../data/Targets/Split_Data_Normalised/Val/SCALED_tmax_targets_val.nc"
var = "TmaxD"
standardise(input_path, output_path, var, mean=target_tmax_mean, max=target_tmax_std)

TEST SET standardisation for targets

In [ ]:
input_path = "../../data/Targets/Split_Data/Test/rhiresd_targets_test.nc"
output_path = "../../data/Targets/Split_Data_Normalised/Test/SCALED_rhiresd_targets_test.nc"
var = "RhiresD"
standardise(input_path, output_path, var, min=target_precip_min, max=target_precip_max)

In [ ]:
input_path = "../../data/Targets/Split_Data/Test/tabsd_targets_test.nc"
output_path = "../../data/Targets/Split_Data_Normalised/Test/SCALED_tabsd_targets_test.nc"
var = "TabsD"
standardise(input_path, output_path, var, mean=target_tabsd_mean, std=target_tabsd_std)

In [ ]:
input_path = "../../data/Targets/Split_Data/Test/tmin_targets_test.nc"
output_path = "../../data/Targets/Split_Data_Normalised/Test/SCALED_tmin_targets_test.nc"
var = "TminD"
standardise(input_path, output_path, var, mean=target_tmin_mean, std=target_tmin_std)

In [ ]:
input_path = "../../data/Targets/Split_Data/Test/tmax_targets_test.nc"
output_path = "../../data/Targets/Split_Data_Normalised/Test/SCALED_tmax_targets_test.nc"
var = "TmaxD"
standardise(input_path, output_path, var, mean=target_tmax_mean, std=target_tmax_std)